# Setup

In [2]:
# load the .env as environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import asyncio
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import BaseMessage
import pdfkit
import markdown
from PyPDF2 import PdfMerger
from tqdm.notebook import tqdm
import nest_asyncio

In [ ]:
nest_asyncio.apply()

# モデルクライアントの作成
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
)

# 各エージェントの作成
writer_agent = AssistantAgent(
    name="WriterAgent",
    model_client=model_client,
    system_message="あなたはWriterAgentです。\n"
    "あなたはPythonとTypeScriptの両方に精通したソフトウェアエンジニアであり、「Python上級者のためのTypeScript」という本を執筆しています。\n"
    "執筆する際には以下の点にちゅういしてください。\n"
    "- 常にTypeScriptの概念や実装をPythonで相当するものと対比して説明しているか。\n"
    "- Pythonの形ヒントやdataclass, Pydanticなどの型ヒントライブラリとTypeScriptの型の比較を行っているか。\n"
    "- 用語の違いや思想の違いなど含めて、Well-definedに記述されているか。\n"
    "-　抽象的な説明の場合でも、実践をイメージしやすいように常にコードとセットで説明しているか。可能な限り、TypeScriptとPython両方のコードを示す。\n"
    "-　内容は網羅的か。\n"
    "- 前のレビュー内容がある場合は、それが適切に反映されているか。"
    
)

reviewer_agent = AssistantAgent(
    name="ReviewerAgent",
    model_client=model_client,
    system_message="あなたはReviewerAgentです。\n"
    "あなたはわかりやすく網羅的な技術書を執筆するエキスパートです。WriterAgentが作成したドラフトをレビューし、技術的な正確性と深さを確認の上、フィードバックを作成してください。\n"
    "また、そのクオリティが十分である場合は、APPROVEと返してください。\n"
    "もし、修正が必要な場合は、WriterAgentにフィードバックを送り、修正を依頼してください。"
    "レビューにあたっては以下の観点に注意してください。\n"
    "- 常にTypeScriptの概念や実装をPythonで相当するものと対比して説明しているか。\n"
    "- Pythonの形ヒントやdataclass, Pydanticなどの型ヒントライブラリとTypeScriptの型の比較を行っているか。\n"
    "- 用語の違いや思想の違いなど含めて、Well-definedに記述されているか。TypeScript特有の用語にはすべて定義が示されているか。\n"
    "-　抽象的な説明の場合でも、実践をイメージしやすいように常にコードとセットで説明しているか。\n"
    "-　内容は網羅的か。\n"
    "- 前のレビュー内容がある場合は、それが適切に反映されているか。"
    "など。"
    
)

# editor_agent = AssistantAgent(
#     name="EditorAgent",
#     model_client=model_client,
#     system_message="あなたはEditorAgentです。レビュー済みのドラフトを編集し、文体や用語の統一を行ってください。"
# )

# 終了条件の設定
termination_condition = TextMentionTermination("APPROVE") | MaxMessageTermination(max_messages=10)

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

# エージェントチームの作成
agent_team = SelectorGroupChat(
    [writer_agent, reviewer_agent],
    model_client=model_client,
    termination_condition=termination_condition,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

# 各章のタイトルリスト
chapter_titles = [
    "TypeScriptの基礎",
    "TypeScriptの型システム",
    "インターフェースと型エイリアス",
    "型推論とユーティリティ型",
    "関数型プログラミングとTypeScript",
    "ジェネリクスと高度な型",
    "TypeScriptにおけるクラスとオブジェクト指向",
    "TypeScriptとPythonの設計思想の違い",
    "Pythonの型ヒントとTypeScriptの型の比較",
    "TypeScriptでの非同期処理",
    "型安全なAPI設計",
    "TypeScriptでのCLIツール開発",
    "フロントエンドとバックエンドの開発",
    "型安全なデータ処理",
    "TypeScriptの型システムの内部構造",
    "TypeScriptの型チェックとコンパイラの仕組み",
    "型安全な開発フロー"
]

# 出力ディレクトリの作成
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

async def main():
    pdf_files = []
    for idx, title in tqdm(enumerate(chapter_titles, start=1), total=len(chapter_titles)):
        print(f"=== {title} ===")
        task_description = f"{title}に関する章の内容を作成してください。"
        stream = agent_team.run_stream(task=task_description)
        chapter_content = ""
        async for message in stream:
            if isinstance(message, BaseMessage):
                print(f"{message.source}: {message.content}")
            else:
                result = message.messages[-2]
                chapter_content += f"{result.content}\n\n"
        
       # Markdownファイルとして保存
        md_filename = os.path.join(output_dir, f"chapter_{idx:02}_{title}.md")
        with open(md_filename, "w", encoding="utf-8") as md_file:
            md_file.write(f"# {title}\n\n")
            md_file.write(chapter_content)
        
        # # MarkdownからHTMLに変換
        # html_content = markdown.markdown(chapter_content, extensions=['fenced_code', 'tables'])
        # html_content = f"<h1>{title}</h1>\n" + html_content
        
        # # HTMLからPDFに変換
        # pdf_filename = os.path.join(output_dir, f"chapter_{idx:02}_{title}.pdf")
        # pdfkit.from_string(html_content, pdf_filename)
        # pdf_files.append(pdf_filename)
        # print(f"Saved {md_filename} and {pdf_filename}\n")


if __name__ == "__main__":
    asyncio.run(main())

  0%|          | 0/17 [00:00<?, ?it/s]

=== TypeScriptの基礎 ===
user: TypeScriptの基礎に関する章の内容を作成してください。
WriterAgent: ## 第1章: TypeScriptの基礎

この章では、TypeScriptの基本的な概念を紹介し、Pythonの対応する要素との比較を行います。TypeScriptはJavaScriptのスーパーセットであり、型安全性やオブジェクト指向プログラミングをサポートしています。以下のセクションで、TypeScriptの基本概念をPythonと対比しながら解説します。

### 1.1 TypeScriptとは？

TypeScriptは、Microsoftが開発したプログラミング言語で、JavaScriptに型システムとその他の機能を追加したものです。TypeScriptの主な目的は、大規模なアプリケーションの開発においてコードの保守性や可読性を向上させることです。

#### Pythonでの相当

Pythonも動的型付け言語でありつつ、型ヒントを用いることでコードの可読性を向上させることができます。型ヒントを用いることによって、IDEによる補完や型チェックも可能になり、ビルド時のエラーを減らすために役立ちます。

### 1.2 基本的な型

TypeScriptは以下の基本的な型をサポートしています：

- `number`
- `string`
- `boolean`
- `any`
- `void`
- `null` / `undefined`

#### Pythonでの相当

Pythonでは、以下のように型ヒントを使って同様のデータ型を表すことができます。

```python
age: int = 25  # number
name: str = "Alice"  # string
is_active: bool = True  # boolean
value: any = None  # any
```

### 1.3 関数の定義

TypeScriptでは、関数の引数や戻り値に型を指定することができます。以下はTypeScriptの例です。

```typescript
function greet(name: string): string {
    return `Hello, ${name}

In [ ]:
# PDFの結合
merged_pdf_path = os.path.join(output_dir, "Complete_Book.pdf")
merger = PdfMerger()
for pdf_file in pdf_files:
    merger.append(pdf_file)
merger.write(merged_pdf_path)
merger.close()
print(f"All chapters have been merged into {merged_pdf_path}")